In [155]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
# Gather training and testing data and process it to be configured for time intervals
from common import *
from sklearn.model_selection import train_test_split
import pandas as pd

nba = get_cleaned_external_data(next_year_salary=True)
# nba = get_cleaned_baseline_data(next_year_salary=True)
FUT_SAL_CLASS = 'future_salary_class'
max = nba[NEXT_Y_SAL].max()
min = nba[NEXT_Y_SAL].min()
nba[FUT_SAL_CLASS] = nba[NEXT_Y_SAL].apply(lambda x: get_salary_class(x,max,min))

nba_full = nba.copy(deep=True)
for i, row in nba.iterrows() :
    next_val = nba.loc[((nba[P_NAME] == row[P_NAME]) & (nba[SZN_START_Y] == (row[SZN_START_Y]+1)))]  
    if not next_val.empty :
        nba.loc[i,'y'] = next_val.index[0]
    else :
        nba.loc[i,'y'] = np.nan
nba = nba.drop(P_NAME, axis=1)

In [157]:

# Normalize the data
print(nba.columns)
# print(y.columns)
feats = nba.columns[1:-3]
feats = list(feats)
orig_feats = feats
feats = feats + [FUT_SAL_CLASS]
base_feats = feats[:15]
print(feats)
print(base_feats)

X = pd.DataFrame(columns=feats)
y = pd.DataFrame()
X = nba.copy(deep=True)
X = X.dropna()
y = nba.loc[X['y']]

X = X[feats]
y = y[feats]
print(X.columns)
print(y.columns)
# Scale the stuff down
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
y_scaled = scaler.transform(y)
y_scaled = y_scaled[:,:(len(base_feats))]

# Sanity check the scaler
print((X['FG']).iloc[0])
print(X_scaled[0, X.columns.get_loc('FG')])
inv = scaler.inverse_transform(X_scaled)
print(inv[0, X.columns.get_loc('FG')])


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.25, random_state=42)

Index(['seasonStartYear', 'MP', 'PTS', 'Age', 'games', 'games_started', 'PER',
       'FTr', 'AST', 'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight',
       'inflationAdjSalary', 'startYear', 'all_star_total', 'all_star_enc',
       'all_nba_enc', 'all_nba_total', 'draft_pick', 'champion',
       'conference_champ', 'mvp', 'mvp_rank', 'mvp_total', 'player_week_enc',
       'player_week_total', 'dpoy', 'dpoy_rank', 'dpoy_total',
       'next_year_salary', 'future_salary_class', 'y'],
      dtype='object')
['MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST', 'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight', 'inflationAdjSalary', 'startYear', 'all_star_total', 'all_star_enc', 'all_nba_enc', 'all_nba_total', 'draft_pick', 'champion', 'conference_champ', 'mvp', 'mvp_rank', 'mvp_total', 'player_week_enc', 'player_week_total', 'dpoy', 'dpoy_rank', 'dpoy_total', 'future_salary_class']
['MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST', 'STL', 'TRB', 'FT', '3P

In [158]:


# Fit the model
from sklearn.neural_network import MLPRegressor
import numpy as np

# MLPClassifier only classifies data as integers or strings, therefore, our problem is one of regression for the neural network
# Consequently I should use the mlp regressor
mlp = MLPRegressor(hidden_layer_sizes=(10,10,10), solver='adam', max_iter=1000)
mlp.fit(X_train,y_train)

# Evaluate the model
import numpy as np
from sklearn.metrics import mean_squared_error
predict_test = mlp.predict(X_test)
test_set_rsquared = mlp.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(predict_test, y_test))
test_set_mse = mean_squared_error(predict_test, y_test)
print('R_squared value: ', test_set_rsquared) # Variances are not nicely correlated
print('RMSE: ', test_set_rmse) # Fairly good at data prediction
print('MSE:', test_set_mse)
y_test

R_squared value:  0.5872203601181804
RMSE:  0.6631353941951874
MSE: 0.43974855103440663


array([[ 0.8784,  0.5664,  1.503 , ...,  0.5788,  1.6408,  0.7872],
       [ 0.8065,  0.8959,  1.2436, ...,  0.9473,  0.8153,  0.1395],
       [ 0.3332,  0.2244,  0.7249, ...,  0.3237, -0.5605,  0.1036],
       ...,
       [ 0.0577, -0.0595, -0.3124, ..., -0.0505, -0.5605, -0.2203],
       [ 0.6994,  0.8628, -0.5718, ...,  0.9643, -1.3859, -0.4721],
       [ 0.4734,  0.2058,  0.7249, ...,  0.1423, -0.0101, -0.2203]])

In [159]:
# Denormalize the data
predict_test = mlp.predict((X_scaled[0]).reshape(1,-1))
print(X_train.shape[1])
print(predict_test)
tmp = np.zeros((1,X_train.shape[1]))
tmp[:,:predict_test.shape[1]] = predict_test
print(tmp)
undone = scaler.inverse_transform(tmp)
print(undone.flatten().tolist()[:13])
print(y[base_feats].iloc[0])

33
[[ 0.2875  0.4635  1.6354 -0.0524  0.6071  0.9263  1.1716 -0.3997 -0.43
   1.3141  0.7797 -0.8396  0.4818  1.2322  1.3888]]
[[ 0.2875  0.4635  1.6354 -0.0524  0.6071  0.9263  1.1716 -0.3997 -0.43
   1.3141  0.7797 -0.8396  0.4818  1.2322  1.3888  0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.    ]]
[1918.1648457266144, 924.3633669738508, 32.51052237546615, 62.33964719642823, 55.460791888233004, 18.835909896068436, 0.5133667970973005, 93.861890134747, 38.01857012030667, 564.002482392147, 227.0830834069988, -1.373276290166352, 346.8850438890649]
MP               848.000
PTS              540.000
Age               35.000
games             26.000
games_started     26.000
PER               23.300
FTr                0.462
AST               28.000
STL               16.000
TRB              265.000
FT               134.000
3P                 0.000
FG               203.000
height            84.000
weight   

In [183]:
# cur_feats should be ndarray and salary_class should be int from 0-10
def predict_player_base(cur_feats, salary_class) :
    x = np.append(cur_feats, salary_class)
    x_scaled = scaler.transform((x).reshape(1,-1))
    pred = mlp.predict((x_scaled).reshape(1,-1))
    tmp = np.zeros((1,X_scaled.shape[1]))
    tmp[:,:pred.shape[1]] = pred
    pred_unscaled = scaler.inverse_transform(tmp)
    return pred_unscaled[:,:pred.shape[1]]

def get_player_external(name, year) :
    tmp = nba_full.loc[((nba_full[P_NAME] == name) & (nba_full[SZN_START_Y] == year))]
    return tmp[orig_feats]

cur_feats = get_player_external('Phil Pressey', 2014)
cur_salary_class = get_salary_class(cur_feats['inflationAdjSalary'], max, min)

print("Phil Pressey Stats 2014")
print(cur_feats.to_string(index=False))
print(max)
print(min)
print(cur_salary_class)

print("Required stats to level up")
print(base_feats)
print(predict_player_base(cur_feats.values, 1))
print(predict_player_base(cur_feats.values, 5))
tmp2 = get_player_external('Phil Pressey', 2015) 
print(tmp2.to_string(index=False))
print(get_salary_class(cur_feats['inflationAdjSalary'], max, min))


'''Column Values: 
    'MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST',
       'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight',
       'inflationAdjSalary', 'startYear', 'all_star_total', 'all_star_enc',
       'all_nba_enc', 'all_nba_total', 'draft_pick', 'champion',
       'conference_champ', 'mvp', 'mvp_rank', 'mvp_total', 'player_week_enc',
       'player_week_total', 'dpoy', 'dpoy_rank', 'dpoy_total',
       'future_salary_class'''
    

Phil Pressey Stats 2014
    MP   PTS  Age  games  games_started  PER   FTr   AST  STL   TRB   FT   3P   FG  height  weight  inflationAdjSalary  startYear  all_star_total  all_star_enc  all_nba_enc  all_nba_total  draft_pick  champion  conference_champ  mvp  mvp_rank  mvp_total  player_week_enc  player_week_total  dpoy  dpoy_rank  dpoy_total
1132.0 213.0 22.0   75.0           11.0  8.8 0.178 241.0 68.0 105.0 29.0 28.0 78.0    71.0   175.0              930738     2014.0               0             0            0              0       10000         0                 0    0    100000          0                0                  0     0     100000           0
41412563.0
20080.0
0
Required stats to level up
['MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST', 'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight']
[[1268.238   439.8141   22.6207   57.8848   17.0063   12.175     0.2097
   248.743    52.4781   38.9637   73.4289   36.8592  150.8297   71.7784
   177.6307]]
[[1748.4447 

c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  def get_baseline_data(file: str) :
c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  def get_baseline_data(file: str) :


"Column Values: \n    'MP', 'PTS', 'Age', 'games', 'games_started', 'PER', 'FTr', 'AST',\n       'STL', 'TRB', 'FT', '3P', 'FG', 'height', 'weight',\n       'inflationAdjSalary', 'startYear', 'all_star_total', 'all_star_enc',\n       'all_nba_enc', 'all_nba_total', 'draft_pick', 'champion',\n       'conference_champ', 'mvp', 'mvp_rank', 'mvp_total', 'player_week_enc',\n       'player_week_total', 'dpoy', 'dpoy_rank', 'dpoy_total',\n       'future_salary_class"

In [ ]:
# Compute the confidence interval of each of the features
from common import *

# Get the unscaled predictions
predict_full = mlp.predict(X_scaled)
tmp = np.zeros(X_scaled.shape)
tmp[:,:predict_full.shape[1]] = predict_full
predicted_unscaled_vals = scaler.inverse_transform(tmp)
predicted_unscaled_vals = predicted_unscaled_vals[:,:predict_full.shape[1]]

# compute the RMSE for each feature in the output vector
actual_unscaled_vals = y[base_feats].values
rmse = np.sqrt(np.square(predicted_unscaled_vals - actual_unscaled_vals))

# compute the percentilse of the RMSE to get the confidence interval of each predicted feature
ci_up = np.percentile(rmse, 95, axis=0) # 95th percentile
ci_low = np.percentile(rmse, 5, axis=0) # 5th percentile
print("Off by at most this much:", ci_up.flatten().tolist())
print(np.max(actual_unscaled_vals, axis=0))
print(np.min(actual_unscaled_vals, axis=0)) 
# print(ci_low.flatten().tolist()) 

# TODO: validate these findings with the very basic network to see if it predicts that these stats will land us in our desired class
np.set_printoptions(suppress=True, precision=4)
print(confidence_interval_numpy(predicted_unscaled_vals, actual_unscaled_vals)) # requires the assumption of gaussianity, but seems to be working?


print(bootstrap_confidence_interval(predicted_unscaled_vals, actual_unscaled_vals)) # should be able to work without the assumption of gaussianity
# computing for rmse, so says that the true root mean squared error has a 95% of landing in these intervals
# residuals are likely far smaller because you have the negatives balancing things out so it is going to be less accurate for computing the confidence interval

residuals = predicted_unscaled_vals - actual_unscaled_vals
ci = np.quantile(residuals, 1 - 0.05, axis=0)
print("HELLO")
print(ci)

# I want to say that my prediction is between these 2 bounds with 95% confidence
# The two functions I provided seem to find the rmse of each parameter with 95% confidence
# the first and 4th method I try do not scale the mean of the data and therefore, I don't think they necessarily apply to giving me the confidence interval I desire



# DO A CASE STUDY on an individual
# ANALYZE THIS MODEL ON BASELINE AND EXTERNAL




Off by at most this much: [1162.6010485438574, 564.5294819959297, 0.9358547475622839, 36.678941065626624, 44.77237141825249, 5.681788518771645, 0.20083569593628162, 176.79757130821858, 50.42570265081218, 279.9271334943758, 140.38598344515225, 67.25290821346142, 211.85565399097388, 2.097975557900953, 17.64002385748879]
[3485.  2832.    40.    85.    83.    45.3    6.   925.   225.  1226.
  756.   402.   978.    90.   325. ]
[  1.    0.   19.    1.    0.  -40.7   0.    0.    0.    0.    0.    0.
   0.   63.  135. ]
[[476.1579 496.692 ]
 [212.2525 222.3542]
 [  0.3576   0.3772]
 [ 13.7566  14.3724]
 [ 18.2243  18.9838]
 [  2.1591   2.2802]
 [  0.072    0.0783]
 [ 56.3487  59.6562]
 [ 18.633   19.5375]
 [ 99.8705 104.9828]
 [ 48.5911  51.2492]
 [ 21.9521  23.1666]
 [ 80.2483  84.0312]
 [  0.7928   0.831 ]
 [  7.0573   7.3689]]
{'column_0': (475.79476645622674, 496.67383016962964), 'column_1': (212.41596990296983, 222.70172029234558), 'column_2': (0.35803130884824563, 0.3767257665685465), '

In [ ]:
# from sklearn.ensemble import RandomForestRegressor 
# random_forest_model = RandomForestRegressor(n_estimators = 60, random_state = 0)

# random_forest_model.fit(X_train, y_train) 

# y_pred = random_forest_model.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("MSE: ",mse)
# print("RMSE: ",rmse)
